# Extract Data from an Online Store
*note: displayed customers and products are fictional*
## Table of Contents
* Imports and obtain access variables
* GraphQL customers and products query
* Clean and export as csv spreadsheets
#### Resource
[Shopify Python API](https://github.com/Shopify/shopify_python_api)

In [1]:
# install shopify graphql framework
!pip install --upgrade ShopifyAPI


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import modules for notebook use
import pandas as pd
import shopify
import urllib.request
import json
from IPython.utils.text import strip_email_quotes

In [18]:
# obtain access variables
access = 'accessenv.json'
# index because file is dict like
access_df = pd.read_json(access, orient='index')
# access each variable and add ''
token = access_df.loc['TOKENSHOPIFY', 0]
#token = f"'{token}'"
endpoint = access_df.loc['ENDPOINTSHOPIFY', 0]
#endpoint = f"'{endpoint}'"
# create merchant variable
merchant = 'store970.myshopify.com'
print('secrets ready')

secrets ready


In [20]:
# define a function to create a connection and authenticate
# current version available at https://shopify.dev/docs/api/admin-graphql#authentication
def shopify_client():
  api_session = shopify.Session(merchant, '2023-04', token) 
  shopify.ShopifyResource.activate_session(api_session)
  client = shopify.GraphQL()
  return client

In [28]:
# define customers query
customers = """
query {
  customers (first: 20) {
    edges {
      node {
        id
        firstName
        lastName
        email
        locale
        createdAt
        updatedAt
        numberOfOrders
        amountSpent {
          amount
          currencyCode
        }
      }
    }
  }
}
"""

In [29]:
# open session and authenticate
client = shopify_client()
# query customers data
customer_data = json.loads(client.execute(customers))

In [30]:
# clean, create dataframe, export as csv
def customers_format(customer_data):
  data = customer_data['data']['customers']['edges']
  flat_data = pd.json_normalize(data)
  column_list = list(flat_data.columns)
  colnames = list(map(lambda x: x.replace('node.', ''), column_list))
  flat_data.columns = colnames
  customer_df = flat_data.copy()
  return customer_df

customers_df = customers_format(customer_data)
customers_df.to_csv('customers_file.csv', index=False)

In [27]:
# define products query
products = """
query {
  productVariants (first: 20) {
    edges {
      node {
        id
        displayName
        title
        sku
        inventoryQuantity
        price
        inventoryItem {
          requiresShipping
        }
      }
    }
  }
  }
  """

In [32]:
# query products data
products_data =  json.loads(client.execute(products))

In [33]:
# clean, create dataframe, export as csv
def products_format(products_data):
  data = products_data['data']['productVariants']['edges']
  flat_data = pd.json_normalize(data)
  column_list = list(flat_data.columns)
  colnames = list(map(lambda x: x.replace('node.', ''), column_list))
  flat_data.columns = colnames
  products_df = flat_data.copy()
  return products_df

products_df = products_format(products_data)
products_df.to_csv('products_file.csv', index=False)

In [7]:
# load csv and show customers data in csv
customers_csv = pd.read_csv('customers_file.csv')
customers_csv

,id,firstName,lastName,email,locale,createdAt,updatedAt,numberOfOrders,amountSpent.amount,amountSpent.currencyCode
0,gid://shopify/Customer/6864981721377,Ayumu,Hirano,ayumu.hirano@example.com,en,2023-03-10T18:11:14Z,2023-03-10T18:11:21Z,2,2859.80,USD
1,gid://shopify/Customer/6864981754145,Russell,Winfield,russel.winfield@example.com,en,2023-03-10T18:11:14Z,2023-03-10T18:11:23Z,4,6656.19,USD
2,gid://shopify/Customer/6864981786913,Karine,Ruby,karine.ruby@example.com,en,2023-03-10T18:11:14Z,2023-03-10T18:11:23Z,1,752.18,USD


In [6]:
# load csv and show products data in csv
products_csv = pd.read_csv('products_file.csv')
products_csv

,id,displayName,title,sku,inventoryQuantity,price,inventoryItem.requiresShipping
0,gid://shopify/ProductVariant/44692793524513,a - S,S,A123,20,1.0,True
1,gid://shopify/ProductVariant/44692793557281,a - M,M,A124,11,1.0,True
2,gid://shopify/ProductVariant/44692793590049,a - L,L,A125,11,1.0,True
3,gid://shopify/ProductVariant/44692812661025,b - Yellow,Yellow,B223,40,2.0,True
4,gid://shopify/ProductVariant/44692812693793,b - Green,Green,B224,21,2.0,True
5,gid://shopify/ProductVariant/44692828389665,c - S,S,C334,60,3.0,True
6,gid://shopify/ProductVariant/44692828422433,c - M,M,C335,31,3.0,True
7,gid://shopify/ProductVariant/44692828455201,c - L,L,C336,31,3.0,True
8,gid://shopify/ProductVariant/44692852834593,d - Black,Black,D434,80,4.0,True
9,gid://shopify/ProductVariant/44692852867361,d - Gold,Gold,D435,40,4.0,True
